# Customers and Products Analysis with SQL

The database analysis was carried out using the DB Browser for SQLite. We find eight tables wich descriptions and foreign keys are shown below:

**Customers**: customer data 

* salesRepEmployeeNumber = employees['employeeNumber']

**Employees**: all employee information 
* reportsTo = employees['employeeNumber']
* officeCode = offices['officeCode']

**Offices**: sales office information

**Orders**: customers' sales orders
* customerNumber = customers['customerNumber']

**OrderDetails**: sales order line for each sales order
* orderNumber = orders['orderNumber']
* productCode = products['productCode']
													 
**Payments**: customers' payment records
* customerNumber = customers['customerNumber']

**Products**: a list of scale model cars
* productLine = productlines['productLine']

**ProductLines**: a list of product line categories

# What we want to know?

There are three main questions we want to answer for this project:
 
* Question 1: Which products should we order more of or less of?
* Question 2: How should we tailor marketing and communication strategies to customer behaviors?
* Question 3: How much can we spend on acquiring new customers?


In [1]:
import sqlite3
conn = sqlite3.connect('stores.db')

In [2]:
%load_ext sql

In [3]:
%sql sqlite:///stores.db

'Connected: @stores.db'

## Question 1

**Which products should we order more of or less of?**

To answer this question, we will use the **'orders'** table to obtain information on each product's total number of orders.

We then worked with the **'orderdetails'** table to determine the top ten selling products.

From these CTEs, we can create a new table that shows the relationship between inventory and product sales quantity.


In [4]:
%%sql 

WITH
qtd_order AS (
SELECT productCode, SUM(quantityOrdered) AS total_orders
  FROM orderdetails
  GROUP BY productCode  
             ),
  
performance AS ( 
SELECT productCode
  FROM orderdetails
 GROUP BY productCode
 ORDER BY SUM(quantityOrdered*priceEach) DESC
 LIMIT 10 
                )
  
SELECT p.productName, q.total_orders, p.quantityInStock, ROUND(1.0*q.total_orders/p.quantityInStock,2) AS low_stock
  FROM products AS p
  JOIN qtd_order AS q
    ON p.productCode = q.productCode
 WHERE p.productCode IN performance
 ORDER BY low_stock DESC;

 * sqlite:///stores.db
Done.


productName,total_orders,quantityInStock,low_stock
1968 Ford Mustang,933,68,13.72
1969 Ford Falcon,965,1049,0.92
1917 Grand Touring Sedan,918,2724,0.34
2001 Ferrari Enzo,1019,3619,0.28
1992 Ferrari 360 Spider red,1808,8347,0.22
1998 Chrysler Plymouth Prowler,986,4724,0.21
1980s Black Hawk Helicopter,1040,5330,0.2
2003 Harley-Davidson Eagle Drag Bike,985,5582,0.18
1952 Alpine Renault 1300,961,7305,0.13
2002 Suzuki XREO,1028,9997,0.1


Classic cars are the most desired.

Regarding the 1968 Ford Mustang, a more extensive stock would be needed as there is a high demand to buy it.

As for cars from the fourth position on the table, it is possible to notice a balanced supply with demand.


## Question 2

**How should we tailor marketing and communication strategies to customer behaviors?**

Let us rank the most-important and least-committed customers; in this way, we can determine how to drive loyalty and attract more customers.

## VIPs

In [5]:
%%sql

WITH 
client_profit AS (
SELECT o.customerNumber,
       SUM(quantityOrdered*(priceEach-buyPrice)) AS profit
  FROM orders AS o
  JOIN orderdetails AS od
    ON o.orderNumber = od.orderNumber
  JOIN products p
    ON od.productCode = p.productCode
 GROUP BY o.customerNumber
                   )

SELECT contactLastName, contactFirstName, city, country
  FROM customers AS c
  JOIN client_profit AS cp
    ON c.customerNumber = cp.customerNumber
 ORDER BY cp.profit DESC 
 LIMIT 5;

 * sqlite:///stores.db
Done.


contactLastName,contactFirstName,city,country
Freyre,Diego,Madrid,Spain
Nelson,Susan,San Rafael,USA
Young,Jeff,NYC,USA
Ferguson,Peter,Melbourne,Australia
Labrune,Janine,Nantes,France


## Least engaged

In [6]:
%%sql

WITH 
client_profit AS (
SELECT o.customerNumber,
       SUM(quantityOrdered*(priceEach-buyPrice)) AS profit
  FROM orders AS o
  JOIN orderdetails AS od
    ON o.orderNumber = od.orderNumber
  JOIN products p
    ON od.productCode = p.productCode
 GROUP BY o.customerNumber
                   )

SELECT contactLastName, contactFirstName, city, country
  FROM customers AS c
  JOIN client_profit AS cp
    ON c.customerNumber = cp.customerNumber
 ORDER BY cp.profit 
 LIMIT 5;

 * sqlite:///stores.db
Done.


contactLastName,contactFirstName,city,country
Young,Mary,Glendale,USA
Taylor,Leslie,Brickhaven,USA
Ricotti,Franco,Milan,Italy
Schmitt,Carine,Nantes,France
Smith,Thomas,London,UK


## Question 3


To determine how much money we can spend acquiring new customers, we can compute the Customer Lifetime Value (LTV), 
which represents the average amount of money a customer generates. We can then determine how much we can spend 
on marketing.

In [7]:
%%sql

WITH 
client_profit AS (
SELECT o.customerNumber,
       SUM(quantityOrdered*(priceEach-buyPrice)) AS profit
  FROM orders AS o
  JOIN orderdetails AS od
    ON o.orderNumber = od.orderNumber
  JOIN products p
    ON od.productCode = p.productCode
 GROUP BY o.customerNumber
                  )
 
SELECT AVG(profit) AS LVT
  FROM client_profit

 * sqlite:///stores.db
Done.


LVT
39039.59438775511
